In [6]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
# SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_REGION")

In [7]:
# Import OpenAI chat model
from langchain.chat_models import ChatOpenAI
import os

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Create the OpenAI chat model
chat = ChatOpenAI(
    model="gpt-3.5-turbo",  # Use "gpt-4" if you want a cheaper alternative
    temperature=0.2
)

s3 clients

In [11]:
import boto3
from langchain.tools import Tool

os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"]=AWS_SECRET_ACCESS_KEY
# os.environ["AWS_REGION"]= AWS_REGION

# Initialize S3 client
s3_client = boto3.client("s3")

# Function to list files in an S3 bucket
def list_s3_objects(bucket_name):
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    if "Contents" in response:
        return [obj["Key"] for obj in response["Contents"]]
    return "No files found in the bucket."

# Function to read a file from S3
def read_s3_object(bucket_name, file_key):
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    content = response["Body"].read().decode("utf-8")
    return content

# Define tools for S3
list_s3_tool = Tool(
    name="ListS3Files",
    func=lambda bucket: list_s3_objects(bucket),
    description="Lists all files in a given S3 bucket."
)

read_s3_tool = Tool(
    name="ReadS3File",
    func=lambda bucket, file_key: read_s3_object(bucket, file_key),
    description="Reads the content of a file from an S3 bucket. Provide bucket name and file key."
)

Agent

In [12]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate


# Define prompt
prompt = ChatPromptTemplate.from_template("{input}")

# Create LLM Agent
agent = create_cohere_react_agent(
    llm=chat,
    tools=[list_s3_tool, read_s3_tool],  # Add S3 tools here
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[list_s3_tool, read_s3_tool],  # Add tools here as well
    verbose=True
)


In [13]:
# Run Agent with a query
response = agent_executor.invoke({"input": "List files in earthquakedb bucket"})
print(response)




> Entering new AgentExecutor chain...


TypeError: Completions.create() got an unexpected keyword argument 'raw_prompting'

In [ ]:
response = agent_executor.invoke({"input": "Read file.txt from my-bucket-name"})
print(response)